# RQ 1. How much time do children spend alone?

In [ ]:
import sys
sys.path.append('/home/nele_pauline_suffo/projects/naturalistic-social-analysis/src')

import pandas as pd
import numpy as np
import sqlite3
import re
from constants import ResearchQuestions
from rq_01_queries import extract_segments_with_buffering

TOTAL_FRAMES = 875_887

In [ ]:
results_df = pd.read_csv(ResearchQuestions.FRAME_LEVEL_INTERACTIONS_CSV)
age_df = pd.read_csv("/home/nele_pauline_suffo/ProcessedData/age_group.csv")

## Face Detection Information

In [4]:
# Calculate mutually exclusive counts
only_child_face = (results_df['face_frame_category'] == 'only_child').sum()
only_adult_face = (results_df['face_frame_category'] == 'only_adult').sum()
both_faces = (results_df['face_frame_category'] == 'both_faces').sum()
no_faces = (results_df['face_frame_category'] == 'no_faces').sum()
analysis_check_face = only_child_face + only_adult_face + both_faces + no_faces - TOTAL_FRAMES

print(f"Frames with ONLY child faces: {only_child_face:,} ({only_child_face / TOTAL_FRAMES * 100:.2f}%)")
print(f"Frames with ONLY adult faces: {only_adult_face:,} ({only_adult_face / TOTAL_FRAMES * 100:.2f}%)")
print(f"Frames with BOTH face types: {both_faces:,} ({both_faces / TOTAL_FRAMES * 100:.2f}%)")
print(f"Frames with NO faces: {no_faces:,} ({no_faces / TOTAL_FRAMES * 100:.2f}%)")
print(f"Analysis check : {analysis_check_face} frames deviation (should be 0)")

Frames with ONLY child faces: 52,395 (5.98%)
Frames with ONLY adult faces: 63,510 (7.25%)
Frames with BOTH face types: 17,967 (2.05%)
Frames with NO faces: 742,015 (84.72%)
Analysis check : 0 frames deviation (should be 0)


## Person Classification

In [5]:
# Calculate the numbers
only_child_person = (results_df['person_frame_category'] == 'only_child').sum()
only_adult_person = (results_df['person_frame_category'] == 'only_adult').sum()
both_persons = (results_df['person_frame_category'] == 'both_persons').sum()
no_persons = (results_df['person_frame_category'] == 'no_persons').sum()
analysis_check_person = only_child_person + only_adult_person + both_persons + no_persons - TOTAL_FRAMES

print(f"Frames with ONLY child persons: {only_child_person:,} ({only_child_person / TOTAL_FRAMES * 100:.2f}%)")
print(f"Frames with ONLY adult persons: {only_adult_person:,} ({only_adult_person / TOTAL_FRAMES * 100:.2f}%)")
print(f"Frames with BOTH person types: {both_persons:,} ({both_persons / TOTAL_FRAMES * 100:.2f}%)")
print(f"Frames with NO persons: {no_persons:,} ({no_persons / TOTAL_FRAMES * 100:.2f}%)")
print(f"Analysis check : {analysis_check_person} frames deviation (should be 0)")

Frames with ONLY child persons: 2,279 (0.26%)
Frames with ONLY adult persons: 9,590 (1.09%)
Frames with BOTH person types: 1,031 (0.12%)
Frames with NO persons: 862,987 (98.53%)
Analysis check : 0 frames deviation (should be 0)


## Combined Face and Person Presence Analysis

In [6]:
# Calculate combined presence patterns using the correct logic
only_child_present = ((results_df['child_present'] == 1) & (results_df['adult_present'] == 0)).sum()
only_adult_present = ((results_df['child_present'] == 0) & (results_df['adult_present'] == 1)).sum()
both_present = ((results_df['child_present'] == 1) & (results_df['adult_present'] == 1)).sum()
no_one_present = ((results_df['child_present'] == 0) & (results_df['adult_present'] == 0)).sum()
analysis_check_combined = only_child_present + only_adult_present + both_present + no_one_present - TOTAL_FRAMES

print(f"Frames with ONLY child present: {only_child_present:,} ({only_child_present / TOTAL_FRAMES * 100:.2f}%)")
print(f"Frames with ONLY adult present: {only_adult_present:,} ({only_adult_present / TOTAL_FRAMES * 100:.2f}%)")
print(f"Frames with BOTH present: {both_present:,} ({both_present / TOTAL_FRAMES * 100:.2f}%)")
print(f"Frames with NO ONE present: {no_one_present:,} ({no_one_present / TOTAL_FRAMES * 100:.2f}%)")
print(f"Analysis check: {analysis_check_combined} frames deviation (should be 0)")

# Additional insights
any_presence = only_child_present + only_adult_present + both_present
print(f"\nSummary Insights:")
print(f"Frames with ANY human presence: {any_presence:,} ({any_presence / TOTAL_FRAMES * 100:.2f}%)")
print(f"Frames with child presence: {only_child_present + both_present:,} ({(only_child_present + both_present) / TOTAL_FRAMES * 100:.2f}%)")
print(f"Frames with adult presence: {only_adult_present + both_present:,} ({(only_adult_present + both_present) / TOTAL_FRAMES * 100:.2f}%)")

Frames with ONLY child present: 53,836 (6.15%)
Frames with ONLY adult present: 70,818 (8.09%)
Frames with BOTH present: 19,438 (2.22%)
Frames with NO ONE present: 731,795 (83.55%)
Analysis check: 0 frames deviation (should be 0)

Summary Insights:
Frames with ANY human presence: 144,092 (16.45%)
Frames with child presence: 73,274 (8.37%)
Frames with adult presence: 90,256 (10.30%)


## Interaction Segments

In [ ]:
segments_df = extract_segments_with_buffering(results_df)
segments_df.to_csv(ReserachQuestions.INTERACTION_SEGMENTS_CSV, index=False)

Creating segments...
Created 3531 segments after buffering.


In [18]:
results_df[(results_df['video_id'] == 25) & (results_df['frame_number'] >= 11650) & (results_df['frame_number'] <= 12100)]

,frame_number,video_id,video_name,has_child_person,has_adult_person,has_child_face,has_adult_face,proximity,child_present,adult_present,speaker,is_audio_interaction,kchi_speech_present,other_speech_present,interaction_category,face_frame_category,person_frame_category
88956,11650,25,quantex_at_home_id255944_2022_03_08_01,0,0,0,1,0.757473,0,1,NaN,False,0,0,Interacting,only_adult,no_persons
88957,11660,25,quantex_at_home_id255944_2022_03_08_01,0,0,0,1,0.699995,0,1,NaN,False,0,0,Co-present Silent,only_adult,no_persons
88958,11670,25,quantex_at_home_id255944_2022_03_08_01,0,0,0,1,1.000000,0,1,NaN,False,0,0,Interacting,only_adult,no_persons
88959,11680,25,quantex_at_home_id255944_2022_03_08_01,0,0,0,0,NaN,0,0,NaN,False,0,0,Alone,no_faces,no_persons
88960,11690,25,quantex_at_home_id255944_2022_03_08_01,0,0,0,0,NaN,0,0,NaN,False,0,0,Alone,no_faces,no_persons
88961,11700,25,quantex_at_home_id255944_2022_03_08_01,0,0,0,0,NaN,0,0,NaN,False,0,0,Alone,no_faces,no_persons
88962,11710,25,quantex_at_home_id255944_2022_03_08_01,0,0,0,0,NaN,0,0,NaN,False,0,0,Alone,no_faces,no_persons
88963,11720,25,quantex_at_home_id255944_2022_03_08_01,0,0,0,1,1.000000,0,1,NaN,False,0,0,Interacting,only_adult,no_persons
88964,11730,25,quantex_at_home_id255944_2022_03_08_01,0,0,0,1,0.742156,0,1,NaN,False,0,0,Interacting,only_adult,no_persons
88965,11740,25,quantex_at_home_id255944_2022_03_08_01,0,0,0,1,0.744798,0,1,NaN,False,0,0,Interacting,only_adult,no_persons


In [9]:
segments_df[segments_df['video_id'] == 25]

,video_id,video_name,category,segment_start,segment_end,start_time_sec,end_time_sec,duration_sec
474,25,quantex_at_home_id255944_2022_03_08_01,Co-present Silent,150,590,5.000000,19.666667,14.666667
475,25,quantex_at_home_id255944_2022_03_08_01,Alone,600,2730,20.000000,91.000000,71.000000
476,25,quantex_at_home_id255944_2022_03_08_01,Alone,2840,3300,94.666667,110.000000,15.333333
477,25,quantex_at_home_id255944_2022_03_08_01,Co-present Silent,3310,3730,110.333333,124.333333,14.000000
478,25,quantex_at_home_id255944_2022_03_08_01,Alone,3740,10970,124.666667,365.666667,241.000000
479,25,quantex_at_home_id255944_2022_03_08_01,Alone,11100,14510,370.000000,483.666667,113.666667
480,25,quantex_at_home_id255944_2022_03_08_01,Interacting,14520,14700,484.000000,490.000000,6.000000


In [9]:
segments_df[segments_df['video_id'] == 25]

,video_id,video_name,category,segment_start,segment_end,start_time_sec,end_time_sec,duration_sec
680,25,quantex_at_home_id255944_2022_03_08_01,Co-present Silent,0,660,0.000000,22.000000,22.000000
681,25,quantex_at_home_id255944_2022_03_08_01,Alone,670,1240,22.333333,41.333333,19.000000
682,25,quantex_at_home_id255944_2022_03_08_01,Interacting,1250,1890,41.666667,63.000000,21.333333
683,25,quantex_at_home_id255944_2022_03_08_01,Alone,1900,2350,63.333333,78.333333,15.000000
684,25,quantex_at_home_id255944_2022_03_08_01,Interacting,2360,3730,78.666667,124.333333,45.666667
685,25,quantex_at_home_id255944_2022_03_08_01,Alone,3740,9760,124.666667,325.333333,200.666667
686,25,quantex_at_home_id255944_2022_03_08_01,Interacting,9770,9940,325.666667,331.333333,5.666667
687,25,quantex_at_home_id255944_2022_03_08_01,Alone,9950,11870,331.666667,395.666667,64.000000
688,25,quantex_at_home_id255944_2022_03_08_01,Interacting,11880,12480,396.000000,416.000000,20.000000
689,25,quantex_at_home_id255944_2022_03_08_01,Alone,12490,13340,416.333333,444.666667,28.333333


In [ ]:
 #--- Step 1: Extract child_id from video_name if not already present ---
def extract_child_id(video_name):
    match = re.search(r'id(\d{6})', video_name)
    return match.group(1) if match else None

segments_df['child_id'] = segments_df['video_name'].apply(extract_child_id)

# Keep only first occurrence of each child_id in age_df
age_df = age_df.drop_duplicates(subset='child_id', keep='first')[['child_id', 'age_at_recording']]

# Ensure both are strings for merging
segments_df['child_id'] = segments_df['child_id'].astype(str)
age_df['child_id'] = age_df['child_id'].astype(str)

# --- Step 2: Aggregate duration per child and category (for filtering) ---
duration_by_child_category = (
    segments_df.groupby(['child_id', 'category'])['duration_sec']
    .sum()
    .unstack(fill_value=0)
)

# Only keep age data for children that have segments
children_with_segments = set(duration_by_child_category.index)
age_df_filtered = age_df[age_df['child_id'].isin(children_with_segments)]

# Sort by age and save to CSV
age_df_filtered = age_df_filtered.sort_values(by='age_at_recording')
age_df_filtered.to_csv(ResearchQuestions.CHILD_ID_CSV, index=False)

In [8]:
import os
import cv2

def get_video_duration(video_path):
    """ Returns the duration of the video in seconds. """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return 0
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    cap.release()
    if fps > 0:
        return frame_count / fps
    return 0

def sum_video_durations(txt_file, main_folder):
    total_duration = 0
    with open(txt_file, 'r') as f:
        for line in f:
            video_name = line.strip()
            video_path = os.path.join(main_folder, f"{video_name}.MP4")
            duration = get_video_duration(video_path)
            total_duration += duration # in seconds
    return total_duration

# Example usage:
txt_file = '/home/nele_pauline_suffo/projects/naturalistic-social-analysis/src/finished_videos.txt'
main_folder = '/home/nele_pauline_suffo/ProcessedData/quantex_videos'
total_duration = sum_video_durations(txt_file, main_folder)
# get total duration in hours, minutes, seconds
hours = int(total_duration // 3600)
minutes = int((total_duration % 3600) // 60)
seconds = int(total_duration % 60)
print(f"Total duration: {hours}h {minutes}m {seconds}s")

Total duration: 81h 32m 11s
